每个输出通道可以识别特定模式

输入通道核识别并组合输入中的模式

1*1卷积，不是别空间模式（不看像素周围），只融合通道。其实就是一个全连接层

In [1]:
import torch
from d2l import torch as d2l

In [2]:
def corr2d_multi_in(X, K):
    # 先遍历X和K的第n个维度（通道维度），再把它们加在一起
    return sum(d2l.corr2d(x, k) for x,k in zip(X, K))

In [3]:
# test
X = torch.tensor([[[0,1,2],[3,4,5],[6,7,8]], [[1,2,3],[4,5,6],[7,8,9]]])
K = torch.tensor([[[0,1],[2,3],], [[1,2],[3,4]]])
corr2d_multi_in(X, K)

tensor([[ 56.,  72.],
        [104., 120.]])

In [12]:
# 多通道的互相关函数
def corr2d_multi_in_out(X, K):
    return torch.stack([corr2d_multi_in(X, k) for k in K], dim=0)

In [8]:
# test torch.stack()
T1 = torch.tensor([[1,2, 100],[3,4, 200], [5, 6, 300]])
print(T1.shape)
T2 = torch.tensor([[5,6, 500],[7,8, 600], [9, 10, 700]])
print(torch.stack([T1, T2], dim=0))
print(torch.stack([T1, T2], dim=0).shape)
print(torch.stack([T1, T2], dim=1))
print(torch.stack([T1, T2], dim=1).shape)
print(torch.stack([T1, T2], dim=2))
print(torch.stack([T1, T2], dim=2).shape)

torch.Size([3, 3])
tensor([[[  1,   2, 100],
         [  3,   4, 200],
         [  5,   6, 300]],

        [[  5,   6, 500],
         [  7,   8, 600],
         [  9,  10, 700]]])
torch.Size([2, 3, 3])
tensor([[[  1,   2, 100],
         [  5,   6, 500]],

        [[  3,   4, 200],
         [  7,   8, 600]],

        [[  5,   6, 300],
         [  9,  10, 700]]])
torch.Size([3, 2, 3])
tensor([[[  1,   5],
         [  2,   6],
         [100, 500]],

        [[  3,   7],
         [  4,   8],
         [200, 600]],

        [[  5,   9],
         [  6,  10],
         [300, 700]]])
torch.Size([3, 3, 2])


In [17]:
# test 1*1 conv

def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.reshape((c_i, h * w))
    K = K.reshape((c_o, c_i))
    Y = torch.matmul(K, X)
    return Y.reshape((c_o, h, w))

X = torch.normal(0, 1, (3, 3, 3))
K = torch.normal(0, 1, (2, 3, 1, 1))

Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)
print(float(torch.abs(Y1 - Y2).sum()) < 1e6)


True
